In [1]:
import requests
import urllib
import pandas as pd
import os
import zipfile
import sqlalchemy as db
import matplotlib.pyplot as plt
import cufflinks as cf
import datetime

In [2]:
cf.set_config_file(offline=True)

# Get Stock Price

In [24]:
def get_all_stock():
    con = db.create_engine("sqlite:///C:/Users/zheji/Desktop/TradingTools/dailyDB.sqlite")
    dfs = []
    for table in con.table_names():
        tmp = pd.read_sql(table, con)
        tmp['ticker'] = table
        tmp['1dayret%'] = (tmp['Adj Close'] - tmp['Adj Close'].shift(1))/tmp['Adj Close'].shift(1) * 100
        tmp['nextdayret%'] = (tmp['Adj Close'].shift(-1) - tmp['Adj Close'])/ tmp['Adj Close'] * 100
        dfs.append(tmp)
    return pd.concat(dfs)
stock_df = get_all_stock()

In [25]:
stock_df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,ticker,1dayret%
0,2018-01-02,67.419998,67.889999,67.339996,67.599998,66.243683,1047800.0,A,NaN
1,2018-01-03,67.620003,69.489998,67.599998,69.320000,67.929161,1698900.0,A,2.544361
2,2018-01-04,69.540001,69.820000,68.779999,68.800003,67.419609,2230700.0,A,-0.750123
3,2018-01-05,68.730003,70.099998,68.730003,69.900002,68.497528,1632500.0,A,1.598821
4,2018-01-08,69.730003,70.330002,69.550003,70.050003,68.644516,1613400.0,A,0.214589


# FINRA DARK POOL VIEW

In [40]:
# load dark pool and lit pool shorting data
con1 = db.create_engine('sqlite:///'+'shortDB'+'.sqlite')
ts = pd.read_sql_query('SELECT * from DARK', con1)
con2 = db.create_engine('sqlite:///'+'shortDB'+'.sqlite')
ts_lit = pd.read_sql_query('SELECT * from Lit', con2)
ts = ts.drop_duplicates()
ts_lit = ts_lit.drop_duplicates()

In [55]:
short = pd.merge(ts, ts_lit, left_on=['Date', 'Symbol'], right_on=['Date', 'Symbol'], how='inner')  # cboe + finra 
view = pd.merge(short, stock_df, left_on=['Date', 'Symbol'], right_on=['Date', 'ticker'], how='inner') # cboe + finra + stock

In [56]:
view['Lit_VolumeRatio'] = view['CBOE_TotalVolume']/view['Volume']
view['Dark_VolumeRatio'] = view['NMS_TotalVolume']/view['Volume']
view.head(5)

,index_x,Symbol,NASDAQCAR_ShortVolume,NASDAQCAR_TotalVolume,NYSE_ShortVolume,NYSE_TotalVolume,Date,NMS_ShortVolume,NMS_TotalVolume,NASDAQCAR_ShortRatio,...,Open,High,Low,Close,Adj Close,Volume,ticker,1dayret%,Lit_VolumeRatio,Dark_VolumeRatio
0,0,A,89508.0,412346.0,36071.0,146290.0,2019-02-20,125579.0,558636.0,0.217070,...,77.529999,78.800003,77.330002,78.559998,77.697357,2076500.0,A,1.341609,0.125743,0.269028
1,5,AAL,844109.0,1372781.0,319835.0,584804.0,2019-02-20,1163944.0,1957585.0,0.614890,...,35.000000,35.240002,34.770000,35.060001,34.596169,6969900.0,AAL,-1.155894,0.165140,0.280863
2,9,AAP,272139.0,469307.0,18423.0,54741.0,2019-02-20,290562.0,524048.0,0.579874,...,166.000000,167.029999,162.589996,166.000000,165.279968,1990200.0,AAP,-0.682054,0.123362,0.263314
3,10,AAPL,2626567.0,5520623.0,1587174.0,3256997.0,2019-02-20,4213741.0,8777620.0,0.475774,...,171.190002,173.320007,170.990005,172.029999,169.807388,26114400.0,AAPL,0.643545,0.121747,0.336122
4,18,ABBV,435128.0,1587068.0,39094.0,260434.0,2019-02-20,474222.0,1847502.0,0.274171,...,80.529999,80.599998,79.139999,79.989998,73.424461,5564100.0,ABBV,-0.658219,0.118325,0.332040


In [ ]:
def single_stock_view(stock):
    df = view[view['Symbol']==stock]
    display_cols = ['Date','Symbol', 'NMS_ShortRatio','CBOE_ShortRatio',  'Dark_VolumeRatio', 'Lit_VolumeRatio', '1dayret%']
    df = df[display_cols]
    df.index = df.Date
    plot_cols = ['NMS_ShortRatio','CBOE_ShortRatio',  'Dark_VolumeRatio', 'Lit_VolumeRatio']
    df[plot_cols].iplot(kind='line')
    
def single_stock_bound_view(stock, plot_cols, window_size, bound, recent_n):
    df = view[view['Symbol']==stock]
    df.index = df.Date
    for col in plot_cols:
        df[col+'_upper'] = df[col].rolling(window_size).mean() + bound * df[col].rolling(window_size).std()
        df[col+'_lower'] = df[col].rolling(window_size).mean() - bound * df[col].rolling(window_size).std()
        df[col+'_mean'] = df[col].rolling(10).mean() 
        df[[col, col+'_mean', col+'_upper', col+'_lower']][-1*recent_n:].iplot(kind='line')
    df['1dayret%'][-1*recent_n:].iplot(kind='bar')

In [75]:
single_stock_view('AMZN')

In [76]:
single_stock_bound_view('AMZN', ['NMS_ShortRatio', 'CBOE_ShortRatio'], 50, 1, 90)

In [45]:
stock_df[stock_df['Date']=='2019-02-20'].head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,ticker,1dayret%
284,2019-02-20,77.529999,78.800003,77.330002,78.559998,77.697357,2076500.0,A,1.341609
284,2019-02-20,35.000000,35.240002,34.770000,35.060001,34.596169,6969900.0,AAL,-1.155894
284,2019-02-20,166.000000,167.029999,162.589996,166.000000,165.279968,1990200.0,AAP,-0.682054
284,2019-02-20,171.190002,173.320007,170.990005,172.029999,169.807388,26114400.0,AAPL,0.643545
284,2019-02-20,80.529999,80.599998,79.139999,79.989998,73.424461,5564100.0,ABBV,-0.658219


In [37]:
# single day cross sectional view (dark+lit+stockdf)
pd.options.display.max_rows = 100  
date = '2020-06-16'
def compute_all_data():
    set1 = set(ts.Date.unique())
    set2 = set(ts_lit.Date.unique())
    set3 = set(stock_df.Date.unique())
    tmp = set1.intersection(set2)
    all_dates = list(tmp.intersection(set3))
    dfs = []
    for date in all_dates:
        print(date)
        dfs.append(day_view(date))
    return pd.concat(dfs)

def day_view(date):
    day_dix = ts[ts['Date'] == date]
    day_lit = ts_lit[ts_lit['Date'] == date]
    day_stock = stock_df[stock_df['Date']== date]
    day_dix = pd.merge(day_dix, day_stock, left_on='Symbol', right_on='ticker', how='inner')
    day_dix = pd.merge(day_dix, day_lit, on='Symbol', how='inner')
    day_dix['Lit_VolumeRatio'] = day_dix['CBOE_TotalVolume'] / day_dix['Volume']
    day_dix['Dark_VolumeRatio'] = day_dix['NMS_TotalVolume'] / day_dix['Volume']
    display_cols = ['Date','Symbol', 'NMS_ShortRatio','CBOE_ShortRatio',  'Dark_VolumeRatio', 'Lit_VolumeRatio', '1dayret%']
    day_dix = day_dix.sort_values('1dayret%', ascending=True)[:99]
    return day_dix[display_cols]
df_ts = compute_all_data()

2020-01-13
2020-05-13
2019-09-10
2019-12-26
2020-01-22
2020-04-21
2019-06-26
2020-05-26
2020-04-01
2020-04-06
2019-08-14
2019-05-24
2020-05-08
2020-01-21
2019-10-15
2020-01-06
2019-03-01
2019-06-25
2019-09-17
2019-12-02
2020-05-07
2020-02-19
2019-09-18
2020-03-19
2020-03-12
2019-11-14
2019-09-13
2019-02-21
2019-12-18
2020-04-13
2020-05-14
2019-12-03
2019-09-26
2020-02-07
2020-06-10
2020-06-08
2019-07-30
2019-09-05
2019-12-11
2020-02-13
2019-10-04
2019-07-15
2020-01-29
2019-12-24
2019-10-25
2020-03-30
2020-02-10
2020-03-10
2019-06-05
2020-01-10
2019-06-20
2019-05-14
2019-10-21
2019-11-29
2019-09-06
2019-06-21
2019-04-11
2019-10-24
2019-04-25
2019-12-13
2019-07-11
2019-08-30
2020-04-14
2020-04-30
2019-05-23
2019-06-04
2019-05-16
2019-05-07
2019-03-14
2019-06-13
2019-03-27
2019-08-28
2019-07-17
2020-02-26
2019-12-10
2019-03-26
2019-12-19
2019-03-12
2019-12-09
2020-01-24
2019-05-21
2020-05-12
2019-02-20
2019-02-28
2020-02-18
2019-06-07
2019-10-29
2019-09-20
2019-05-01
2019-05-02
2019-07-18

,Date,Symbol,NMS_ShortRatio,CBOE_ShortRatio,Dark_VolumeRatio,Lit_VolumeRatio,1dayret%
5,2020-01-13,ABMD,0.531569,0.537607,0.384272,0.198418,-11.086423
197,2020-01-13,ILMN,0.586464,0.644485,0.320378,0.113637,-3.689686
34,2020-01-13,ANTM,0.542148,0.643068,0.282089,0.107523,-3.574451
80,2020-01-13,CI,0.370286,0.526803,0.240645,0.149002,-3.219517
374,2020-01-13,UNH,0.357486,0.533213,0.375317,0.142432,-3.144372
...,...,...,...,...,...,...,...
232,2020-03-09,IVZ,0.627306,0.423099,0.359509,0.110529,-11.661583
286,2020-03-09,MGM,0.369965,0.478149,0.145604,0.140452,-11.660082
416,2020-03-09,TSCO,0.443854,0.381757,0.273888,0.146135,-11.630965
368,2020-03-09,RJF,0.661297,0.612352,0.248660,0.152231,-11.589668


In [7]:
def stock_ts_view(stock):
    
interest = ['AAPL', 'QQQ', 'W', 'SHOP', 'UAL', 'MGM', 'DAL', 'SAVE', 'AMD', 'WFC', 'SPY', 'MSFT', 'GSX', 'PDD']
ts_dix = pd.DataFrame()
ts.index = ts.Date
for symbol in interest:
    ts_dix[symbol] = ts[ts['Symbol']==symbol].NMS_ShortRatio
ts_dix[-30:].iplot(kind='line')

# Lit/Dark Comparison

In [14]:
df = pd.DataFrame()
ticker = 'SHOP'
df['Lit'] = ts_lit[ts_lit['Symbol']==ticker].CBOE_ShortRatio
df['Dark'] = ts[ts['Symbol']==ticker].NMS_ShortRatio
print(df.corr())
df.iplot( kind='line')

           Lit      Dark
Lit   1.000000  0.587425
Dark  0.587425  1.000000


# Process Cboe Detailed Short Sales Data

In [ ]:
def load_stock_data(num_days = 9999):
    output = []
    for s in con.table_names():
        tmp = pd.read_sql(s, con)[-1*num_days:]
        tmp['Ticker'] = s
        output.append(tmp)
    stock_df = pd.concat(output)
    stock_df.index = stock_df['Ticker']
    del stock_df['Ticker']
    return stock_df

def analyze_short_volume(df, day_df):
    print(df.head(5))
    df['value'] = df['Size'] * df['Price']
    summary = pd.DataFrame()
    other = []
    groupdf = df.groupby('Symbol')
    summary['total_short_volume'] = groupdf['Size'].sum()
    summary['total_trades'] = groupdf['Time'].count()
    summary['average_price'] = groupdf['value'].sum()/summary['total_short_volume']
    summary['largest_block'] = groupdf['Size'].max()
    summary['median_size'] = groupdf['Size'].median()
    summary['max'] = df.iloc[groupdf['Size'].idxmax(), 1]
    
    summary = pd.merge(summary, day_df, left_index = True, right_on = 'Ticker')
    summary['day_mid_price'] = (summary['Open'] + summary['Close'] + summary['High'] + summary['Close'])/4
    print(summary.head(5))
    #summary = summary[summary.index.isin(con.table_names())]
    
    #load stock data
#     stock_df = load_stock_data(1)
#     summary = pd.concat([stock_df, summary], axis=1 )
#     summary = summary.dropna()
#     print(summary[['Open', 'High', 'Low', 'Close']].mean())
#     summary['stock_median_price'] = summary[['Open', 'High', 'Low', 'Close']].mean()
#     summary['short_vol_ratio'] = summary['total_short_volume']/summary['Volume'] * 100 
#     display_cols = ['Adj Close', 'total_short_volume', 'short_vol_ratio', 'stock_median_price', 'average_price']
#     return summary[display_cols]

def load_all_stock():
    dfs = []
    for s in con.table_names():
        tmp = pd.read_sql(s, con)
        tmp['Ticker'] = s
        dfs.append(tmp)
    return pd.concat(dfs)

In [ ]:
stock_df = load_all_stock()

In [ ]:
date='2020-04-20'
exchanges = ['EDGA', 'EDGX', 'BYX', 'BZX']
datestr = ''.join(date.split('-'))
con = db.create_engine('sqlite:///'+'yahooData'+'.sqlite')
day_stock = stock_df[stock_df['Date'] == date]
exch_dfs = []
for exch in exchanges:
    with open('DIX_data/'+exch+datestr+'.txt') as f:
        next(f)
        output = []
        for line in f:
            token = line.split('|')
            output.append({'Symbol': token[1],
                           'Time': token[3],
                           'Size': int(token[5]),
                           'Price': float(token[6]),
                           })
        df = pd.DataFrame(output)
        df['exchange'] = exch
        exch_dfs.append(df)

all_df = pd.concat(exch_dfs)

In [ ]:
all_df

In [ ]:
all_df['value'] = all_df['Size'] * all_df['Price']
summary = pd.DataFrame()
other = []
groupdf = all_df.groupby('Symbol')
summary['total_short_volume'] = groupdf['Size'].sum()
summary['total_trades'] = groupdf['Time'].count()
summary['average_price'] = groupdf['value'].sum()/summary['total_short_volume']
summary['largest_block'] = groupdf['Size'].max()
summary['median_size'] = groupdf['Size'].median()
summary = pd.merge(summary, day_stock, left_index = True, right_on = 'Ticker', how = 'inner')



In [ ]:
summary['short_ratio'] = summary['total_short_volume'] / summary['Volume']
summary['day_mid_price'] = (summary['Open'] + summary['Close'] + summary['High'] + summary['Close'])/4
summary.sort_values('short_ratio', ascending=False)

In [ ]:
def load_all_stock():
    dfs = []
    for s in con.table_names():
        dfs.append(pd.read_sql(s, con))
    return pd.concat(dfs)

def analyze_short_volume(df):
    df['value'] = df['Size'] * df['Price']
    summary = pd.DataFrame()
    other = []
    groupdf = df.groupby('Symbol')
    summary['total_short_volume'] = groupdf['Size'].sum()
    summary['total_trades'] = groupdf['Time'].count()
    summary['average_price'] = groupdf['value'].sum()/summary['total_short_volume']
    summary['largest_block'] = groupdf['Size'].max()
    summary['median_size'] = groupdf['Size'].median()
    summary['max'] = groupdf['Size'].idxmax()
    summary = summary[summary.index.isin(con.table_names())]
    
    #load stock data
    stock_df = load_stock_data(1)
    summary = pd.concat([stock_df, summary], axis=1 )
    summary = summary.dropna()
    print(summary[['Open', 'High', 'Low', 'Close']].mean())
    summary['stock_median_price'] = summary[['Open', 'High', 'Low', 'Close']].mean()
    summary['short_vol_ratio'] = summary['total_short_volume']/summary['Volume'] * 100 
    display_cols = ['Adj Close', 'total_short_volume', 'short_vol_ratio', 'stock_median_price', 'average_price']
    return summary[display_cols]